In [3]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [8]:
from utils import downloadData
from utils import dataProc
download = downloadData.downloadData()
download.getData()
data = dataProc.dataProc()
df = data.loadData()
df_dict = data.cleanData(df)

Scrapping started
Scrapping finished
Filenames
['2020ago', '2020jul', '2020jun', '2020may', '2020abr', '2020mar', '2020ene', '2019', '2018', '2017', '2015', '2014', '2013']
Files downloaded
list files
['./data\\2013.xlsx', './data\\2014.xlsx', './data\\2015.xlsx', './data\\2017.xlsx', './data\\2018.xlsx', './data\\2019.xlsx', './data\\2020abr.xlsx', './data\\2020ago.xlsx', './data\\2020ene.xlsx', './data\\2020jul.xlsx', './data\\2020jun.xlsx', './data\\2020mar.xlsx', './data\\2020may.xlsx']
years files
[WindowsPath('2013.xlsx'), WindowsPath('2014.xlsx'), WindowsPath('2015.xlsx'), WindowsPath('2017.xlsx'), WindowsPath('2018.xlsx'), WindowsPath('2019.xlsx'), WindowsPath('2020ago.xlsx')]
Loading data
Loading data finished
Cleanning data
Cleaning data finsihed


In [ ]:
#b = cleanData.cleanData()
#df_dict = b.cleanData()

In [9]:
df_dict.keys()

dict_keys(['2013', '2014', '2015', '2017', '2018', '2019', '2020'])

In [10]:
df_dict['2017']

,departamento,municipio,operadora,contrato,campo,enero,febrero,marzo,abril,mayo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,antioquia,PUERTO NARE,ecopetrol s.a.,operacion-directa ecopetrol,area teca-cocorna,1230.97,1108.93,1026.16,1035.77,1046.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,antioquia,PUERTO NARE,mansarovar energy colombia ltd,nare asociacion,nare,271.742,257.036,252.903,232.9,261.774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,antioquia,PUERTO NARE,mansarovar energy colombia ltd,nare asociacion,underriver,935.677,796.929,880.226,899.867,993.742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,antioquia,PUERTO TRIUNFO,ecopetrol s.a.,operacion-directa ecopetrol,area teca-cocorna,393,386,361.613,365.433,395.613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,antioquia,YONDO,ecopetrol s.a.,casabe,casabe,12745,12535.2,12756.4,12720.3,12491.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,tolima,PURIFICACIÓN,hocol s.a.,chenche hocol,chenche,59.5484,55.0357,53.6774,51.9667,51.2903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,tolima,PURIFICACIÓN,hocol s.a.,espinal - hocol,matachín norte,2219.45,2201.07,2156.74,2171.57,2131.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,tolima,PURIFICACIÓN,hocol s.a.,espinal - hocol,matachín sur,359.323,348.536,336.871,309.4,298.097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,tolima,PURIFICACIÓN,hocol s.a.,espinal - hocol,purificación,129.645,126.75,124.226,121.5,120.742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
January= df_dict['2020'].columns.get_loc("enero")
df_dict['2020']['Total_Prod']=df_dict['2020'].iloc[:,January::].fillna(0).sum(axis=1)

In [ ]:
df_dict['2020'].isnull().sum()

In [ ]:
#Eliminar duplicados, si los hay
df_dict['2020']=df_dict['2020'].drop_duplicates()

In [ ]:
df_dict['2020']

In [ ]:
Highest_Prod=df_dict['2020'].groupby('campo')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
#Seleccionar los campos con mayor producción 
Highest_Prod.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:5]

In [ ]:
Op_Field_2018=df_dict['2018'].groupby(['operadora','departamento'])['campo'].count().reset_index(name ='#Prod_Fields')
display(Op_Field_2018[(Op_Field_2018['departamento']=='CASANARE')&(Op_Field_2018['#Prod_Fields']>5)].count())
display(Op_Field_2018[(Op_Field_2018['departamento']=='CASANARE')&(Op_Field_2018['#Prod_Fields']>5)])

In [ ]:
Jan_2018= df_dict['2018'].columns.get_loc("enero")
df_dict['2018']['Total_Prod']=df_dict['2018'].iloc[:,Jan_2018::].fillna(0).sum(axis=1)
Cont_Highest_Prod= df_dict['2018'].groupby('contrato')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
Cont_Highest_Prod.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:5]

In [ ]:
Op_High_Prod_2019= df_dict['2019'].groupby('operadora')['agosto'].sum().reset_index(name ='Prod_Aug')
Op_High_Prod_2019.sort_values(by=['Prod_Aug'], ascending=False).iloc[0:10]